# Demand Analysis I

## Preliminaries

In [25]:
# Load libraries
import numpy as np
import pandas as pd
import HWDataProcessing as dp
import EstimationTools as et
from EstimationTools import Model
from EstimationTools import ModelSpecification

In [26]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [37]:
a.reshape((9,),order='F')

f = lambda i: [i%3,i//3]
f(2) 


[2, 0]

In [34]:
b = a.reshape((9,),order='F')
b[[2,4]] = 0
b.reshape((3,3),order='F')

array([[1, 2, 3],
       [4, 0, 6],
       [0, 8, 9]])

In [2]:
# Load data
dataFile = "data/Shining32.csv"
data = pd.read_csv(dataFile)

# Process data
data = dp.CleanDataSet(data)

# Take a look
data.head()

,buyerid,age,education,inc,gender,location,purchase,brandid,mint,white,...,loc_Queens,gen_Female,gen_Male,gen_Other,purchase_InStore,purchase_Online,brand_Aquafresh,brand_Colgate,brand_Crest,brand_Sensodyne
0,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,1,1,...,0,1,0,0,0,1,0,1,0,0
1,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,1,0,...,0,1,0,0,0,1,0,1,0,0
2,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,0,1,...,0,1,0,0,0,1,0,1,0,0
3,1,56,College,0.079523,Female,Brooklyn,Online,Crest,1,1,...,0,1,0,0,0,1,0,0,1,0
4,1,56,College,0.079523,Female,Brooklyn,Online,Crest,0,0,...,0,1,0,0,0,1,0,0,1,0


## Simple logit model estimation

In [ ]:
# Model specification
J = 10     # number of products
yName = "Chosen"
xName = ["mint","white","fluoride","kids"]  # product characteristics
zName = []              # consumer attributes
pName = "priceperpack"  # product price
brands = "brandid"      # brand identifier
iv = ["discount"]

logitMod = Model(type='logit',data=data,J=J,y=yName,x=xName,z=zName,
                p=pName,brands = brands,iv=iv)

In [ ]:
estimates,vcv = logitMod.fitLogit()
estimates,vcv,np.sqrt(np.diag(vcv))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 222
         Gradient evaluations: 37
         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 144
         Gradient evaluations: 22


(array([ 1.89389967, -1.0634944 ,  0.28448555, -1.02661954, -2.31469652]),
 array([[ 0.15988221,  0.05801044,  0.03512713,  0.16004204, -0.17914266],
        [ 0.05801044,  0.02248345,  0.01362962,  0.05817277, -0.06816386],
        [ 0.03512713,  0.01362962,  0.00876988,  0.03496936, -0.04010268],
        [ 0.16004204,  0.05817277,  0.03496936,  0.16666963, -0.18128019],
        [-0.17914266, -0.06816386, -0.04010268, -0.18128019,  0.2281882 ]]),
 array([0.39985273, 0.14994481, 0.09364762, 0.40825192, 0.47769049]))

In [ ]:
elasticities= logitMod.getElasticityLogit(estimates)
elasticities

array([[ -8.76876708,   2.90049744,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,  -6.7007548 ,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744, -10.92671049,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,  -8.81924594,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
        -10.53720839,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
          0.09924563, -10.00598566,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  

## Mixed logit model estimation

In [3]:
# Model specification
modelSpec = ModelSpecification()
modelSpec.type = 'mixed_logit'
modelSpec.data = data
modelSpec.J = 10     # number of products
modelSpec.i = "buyerid" # consumer identifier
modelSpec.y = "Chosen" # choice variable
modelSpec.y2 = "Chosen" # choice variable
modelSpec.x = ["mint","white","fluoride","kids"]  # product characteristics
modelSpec.p = "priceperpack"  # product price
modelSpec.iv = ["discount"]       # iv for prices
modelSpec.zeta = ["income"]              # obs. consumer attributes

# Second moment interactions: choose which product characteristics (X) and 
# consumer attributes to interact (zeta) to form first-choice moments
# It must be defined as a list of index pairs [X,zeta]
modelSpec.XZetaInter = [[4,0]] # X=4(=len(x)) -> price; zeta=0 -> income 

# Third moment interactions: choose which product characteristics of first- and
# second-choice to interact to form second-choice momentes
# It must be defined as a list of indexes
modelSpec.X1X2Inter = [4] #X=4 -> interact price

# unobs. consumer attributes. It is a kx1 vector, where k = len([X,p]), or 0s 
# and 1s. A 1 in entry k indicates that product characteristic k is interacted with
# an unobserved consumer attribute.
modelSpec.nu = np.array([1,1,1,1,1])
modelSpec.ns = 1000   # number of draws for Monte-Carlo integration
modelSpec.seed = 1984 # seed for random number generation

modelSpec.secondChoice = True # Whether second choice moments are used in estimation
modelSpec.brands = "brandid"      # brand name variable

# Model instance creation
mixedLogitMod = Model(modelSpec)

In [10]:
a = ['sds', 'sdds', 'sd']
b = np.array([1,0,1]) == 1


TypeError: only integer scalar arrays can be converted to a scalar index

In [5]:
elasticities= logitMod.getElasticityLogit(estimates)
elasticities

array([[ -8.76876708,   2.90049744,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,  -6.7007548 ,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744, -10.92671049,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,  -8.81924594,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
        -10.53720839,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
          0.09924563, -10.00598566,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  